In [17]:
from dataclasses import dataclass
import math

@dataclass
class StockPosition:
    stock_code: str
    shares: int  # Assuming this should be an integer.
    current_price: float
    current_percentage: float = None  # This will be calculated if not provided.
    desired_percentage: float = None  # Must be provided before adjustment.

@dataclass
class StockAdjustment:
    stock_code: str
    shares_to_adjust: int  # Positive for buy, negative for sell.

def calculate_total_market_value(portfolio):
    """Calculate the total market value of the portfolio."""
    return sum(position.shares * position.current_price for position in portfolio)

def calculate_current_percentages(portfolio, total_market_value):
    """Calculate and set the current percentage for each stock position if not already set."""
    for position in portfolio:
        if position.current_percentage is None:
            position.current_percentage = (position.shares * position.current_price) / total_market_value

def adjust_position(portfolio, portfolio_cash):
    """
    Adjust the stock positions in the portfolio to match the desired allocation percentages.
    
    :param portfolio: List[StockPosition] - The current positions in the portfolio.
    :param portfolio_cash: float - The available cash for adjustment.
    :return: Tuple[List[StockAdjustment], float] - Adjustments needed and remaining cash after adjustments.
    """

    # Calculate initial total market value including cash
    total_market_value = calculate_total_market_value(portfolio) + portfolio_cash
    
    # Calculate and set current percentages where necessary
    calculate_current_percentages(portfolio, total_market_value)
    
    adjustments = []
    
    # Perform adjustments calculations
    for position in portfolio:
        if position.desired_percentage is None:
            raise ValueError(f"No desired percentage set for {position.stock_code}")
        
        desired_value = total_market_value * position.desired_percentage
        current_value = position.shares * position.current_price
        
        value_difference = desired_value - current_value
        
        shares_to_adjust = math.floor(value_difference / position.current_price)
        
        adjustment_cost = shares_to_adjust * position.current_price
        
        # Update remaining cash after considering this adjustment.
        portfolio_cash -= adjustment_cost
        
        adjustments.append(StockAdjustment(stock_code=position.stock_code, shares_to_adjust=shares_to_adjust))
        
    return adjustments, portfolio_cash

# Example usage:
portfolio = [
   StockPosition(stock_code='AAPL', shares=133, current_price=150, desired_percentage=0.50),
   StockPosition(stock_code='TSLA', shares=22, current_price=720, desired_percentage=0.30),
   # More positions can be added here...
]

adjustments, remaining_cash= adjust_position(portfolio, 0)

print("Adjustments:")
for adjustment in adjustments:
    print(adjustment)
print("Remaining Cash:", remaining_cash)

Adjustments:
StockAdjustment(stock_code='AAPL', shares_to_adjust=-14)
StockAdjustment(stock_code='TSLA', shares_to_adjust=-8)
Remaining Cash: 7860
